In [3]:
import json
import websocket
from numpy import array
from tqdm import tqdm_notebook as tqdm
from bitstamp import *
import os
from joblib import dump, load

from stock_agent import *


import json
import io


In [4]:
def load_online(minutes, window, val_end = 1626388860):
    tec = TecAn(windows = window, windows_limit = 100)
    sourceDataGenerator = SourceDataGenerator(tec = tec)


    online = OnLineDataProvider(
                 sourceDataGenerator = sourceDataGenerator,
                 minutes = minutes,
                 train_keys = ["btcusd"],
                 train_limit = 40,
                 val_limit = 400,
                 val_keys = ["btcusd"],
                 #val_start = val_start,
                 val_end = val_end,
                 train_start_list = [1626386301]
    )

    online.load_cache()
    return online

In [5]:

model = load('model/min-3-window-20')

In [1]:
back = BackTest(value = 100, 
                    verbose = True, 
                    sell_on_profit = True,
                    pending_sell_steps = 1)


request_sell = lambda price: back.request_sell(price)
request_buy = lambda price: back.request_buy(price)
on_state = lambda timestamp, price: back.on_state(timestamp, price)

stock = StockAgent(
    model = model,
    request_sell = request_sell,
    request_buy = request_buy,
    on_state = on_state
)


NameError: name 'BackTest' is not defined

In [4]:
on_new_data = lambda x: print(x)
on_new_data = lambda x: stock.on_x(x)

on_state = lambda timestamp, price, buy, sell: print("{} {} {} {}".format(timestamp, price, buy, sell))
on_state = lambda timestamp, price, buy, sell: stock.on_new_state(timestamp, price, buy, sell)

agent = DataAgent(
    on_state = on_state,
    on_new_data = on_new_data
)

TIMESTAMP_KEY = "timestamp"

raw_data_live = []


class RawStateDownloader(LiveBitstamp):
    
    def __init__(self, on_raw_data = lambda raw: print(raw)):
        self.trade = {}
        self.on_new_data_count = 0
        self.on_raw_data = on_raw_data
                
    def process(self, raw):
        raw_data_live.append(raw)
        self.on_raw_data(raw)
        timestamp = raw[TIMESTAMP_KEY]
        time = pd.to_datetime(timestamp, unit='s')
        #print(f'{time}: {agent.on_new_data_count} {len(raw_data_live)}')
        

Resample 2Min - TecAn ( windows 20, windows_limit 100 )


In [5]:

on_raw_data = lambda raw: agent.on_new_raw_data(raw)


live = RawStateDownloader(on_raw_data = on_raw_data)


In [6]:
back.reset()

bt = Bitstamp(live, currency = "btcusd")

bt.connect()

Bitstamp Websocket Opened.
Reading form live_trades_btcusd and order_book_btcusd
2021-07-06 08:44:07 Buy (34852.51): ask: 34852.51
2021-07-06 08:46:09 Sell (34885.53) 33.0199999999968 total: 100.09474210035374
2021-07-06 09:02:05 Buy (34911.99): ask: 34911.99
2021-07-06 09:10:06 Sell (34935.67) 23.68000000000029 total: 100.16256980080851
2021-07-06 09:12:07 Buy (34736.16): ask: 34736.16

Bitstamp Websocket Closed.


In [7]:
back.report()

100
0.16256980080851235
0.16256980080851235
